In [4]:
from playwright.async_api import async_playwright
import nest_asyncio
nest_asyncio.apply()

In [20]:
async def scrape_tweet(url: str)->dict:
    _xhr_calls = []

    async with async_playwright() as pw:

        async def intercept_response(response):
            if response.request.resource_type == "xhr":
                _xhr_calls.append(response)

        browser = await pw.chromium.launch(headless=True)
        #context =  browser.new_context(viewport={"width":1920, "height":1000})
        page = await browser.new_page()
        
        await page.goto(url)
        # enable background request intercepting:
        page.on("response", intercept_response)
        # go to url and wait for the page to load
        
        await page.wait_for_selector("[data-testid='tweet']")

        # find all tweet background requests:
        tweet_calls = [f for f in _xhr_calls if "TweetResultByRestId" in f.url]
        for xhr in tweet_calls:
            data = await xhr.json()
            return data["data"]["tweetResult"]["result"]

In [21]:
print(await scrape_tweet("https://twitter.com/Scrapfly_dev/status/1664267318053179398"))

{'__typename': 'Tweet', 'rest_id': '1664267318053179398', 'core': {'user_results': {'result': {'__typename': 'User', 'id': 'VXNlcjoxMzEwNjIzMDgxMzAwNDAyMTc4', 'rest_id': '1310623081300402178', 'affiliates_highlighted_label': {}, 'is_blue_verified': True, 'profile_image_shape': 'Circle', 'legacy': {'created_at': 'Mon Sep 28 16:51:22 +0000 2020', 'default_profile': True, 'default_profile_image': False, 'description': 'Web Scraping API - turn any website into a database!\n\nScrapFly allows you to quickly achieve your data goals without web scraping challenges and errors.', 'entities': {'description': {'urls': []}, 'url': {'urls': [{'display_url': 'scrapfly.io', 'expanded_url': 'https://scrapfly.io', 'url': 'https://t.co/1Is3k6KzyM', 'indices': [0, 23]}]}}, 'fast_followers_count': 0, 'favourites_count': 35, 'followers_count': 218, 'friends_count': 970, 'has_custom_timelines': True, 'is_translator': False, 'listed_count': 2, 'location': 'Paris', 'media_count': 28, 'name': 'Scrapfly', 'norma